In [1]:
import mido
import glob
import numpy as np
from scipy.sparse import lil_matrix
from sklearn.preprocessing import normalize

# Die Eingabedaten

Die midi-Dateien wurden von <a href="http://www.piano-midi.de/beeth.htm" target="blank">piano-midi.de</a> herunter geladen und im Unterordner /Data/Beethoven/ gespeichert.

Wir benutzen das Python Paket <a href="https://mido.readthedocs.io/en/latest/index.html" target="blank">mido</a> um midi-Dateien zu öffnen, zu verändern und zu speichern. Der Befehl

```python
mido.MidiFile("./path/to/file.mid")
```
öffnet file.mid als Python Objekt. Wir können nun durch das Objekt iterieren und so nach und nach alle midi-Nachrichten bekommen.

In [2]:
elise = "./Data/Beethoven/elise.mid"
elise_midi = mido.MidiFile(elise)

for counter,message in enumerate(elise_midi):
    if counter >=50:
        break
    
    print(counter,message,message.time)

0 <meta message track_name name='Für Elise' time=0> 0
1 <meta message copyright text='Copyright © 2004 by Bernd Krueger' time=0> 0
2 <meta message text text='Ludwig van Beethoven' time=0> 0
3 <meta message text text='Poco moto' time=0> 0
4 <meta message text text='Fertiggestellt am 9.10.2004\n' time=0> 0
5 <meta message text text='Update am 30.8.2012\n' time=0> 0
6 <meta message text text='Dauer: 2:48 Minuten\n' time=0> 0
7 <meta message smpte_offset frame_rate=25 hours=32 minutes=0 seconds=3 frames=0 sub_frames=0 time=0> 0
8 <meta message time_signature numerator=3 denominator=8 clocks_per_click=12 notated_32nd_notes_per_beat=8 time=0> 0
9 <meta message key_signature key='C' time=0> 0
10 <meta message set_tempo tempo=867303 time=0> 0
11 <meta message track_name name='Piano right' time=0> 0
12 program_change channel=0 program=0 time=0 0
13 control_change channel=0 control=7 value=100 time=0 0
14 control_change channel=0 control=10 value=64 time=0 0
15 control_change channel=0 control=9

# Simplere Darstellung

Um einfacher mit den Noten zu arbeiten, überführen wir sie in eine simplere Darstellung. Hierbei werden alle Notenwerte in der Reihenfolge des Auftretens im Stück in einer Liste gespeichert.


In [3]:
#midi in noten-Liste umwandeln
def midi2note_sequence(midi):
    sequence = []
    
    for message in midi:
        if not message.is_meta:
            if message.type == 'note_on':
                if message.velocity > 0:
                    sequence.append(message.note)
   
    return sequence

In [4]:
#noten-Liste in midi umwandeln
def note_sequence2midi(sequence, note_length):
    output_midi = mido.MidiFile()
    track = mido.MidiTrack()
    output_midi.tracks.append(track)

    track.append(mido.Message('program_change', program=0))

    for note in sequence:
        track.append(mido.Message('note_on', 
                                  note = note, 
                                  time = note_length))
        track.append(mido.Message('note_off', 
                                  note = note, 
                                  time = note_length))

    return output_midi

Mit diesen Methoden lassen sich die midi-Objekte in Listen von Notenwerten und wieder zurück umwandeln.

Diese Umwandlung wenden wir nun auf elise an.

In [5]:
note_length = 75

sequence = midi2note_sequence(elise_midi)

midi_out = note_sequence2midi(sequence,note_length)
midi_out.save("./Results/elise_simple.mid")

# Markow-Ketten

Unter Markow-Ketten versteht man Stochastische Prozesse, bei denen die Wahrscheinlichkeiten für das Auftreten zuküftiger Ereignisse, basierend auf dem derzeitigen Zustand, angegeben werden.

Dies kann dazu genutzt werden um Sequenzen von Ereignissen des selben Prozesses vorherzusagen, indem, beginnend mit einem gewählten Ereigniss, immer wieder das nächste Ereigniss, basierend auf dem zuletzt gewählten, "ausgewürfelt" wird.

## Beispiel: Wetter
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Elementarymarkow.png/280px-Elementarymarkow.png" />
Benutzen wir eine Markow-Kette um eine einfache Wettervorhersage zu tätigen. Das Wetter wird wie folgt kodiert:

1 = "sonnig", 2 = "bewölkt", 3 = "regnerisch"

Wir nehmen an, dass in 80% der Fälle auf Sonne Regen folgt und in 20% der Fälle es nach Sonne bewölkt ist. Außerdem, dass die Wahrscheinlich keit für Sonne und Regen nach einem bewölkten Tag 50/50 ist und dass in 10% der Fälle Sonne auf Regen folgt und es in 90% der Fälle nach Regen bewölkt ist.

Somit ergibt sich die sogenannte Übergangsmatrix mit den Übergangswahrscheinlichkeiten:

 Matrix  | sonnig | bewölkt | regnerisch
 ---|:---:|:---:|:---:
**sonnig** | 0 | 0,2 | 0,8
**bewölkt** |0,5 | 0 | 0,5
**regnerisch** | 0,1 | 0,9 | 0

Somit könnte sich diese Wettersequenz ergeben:

sonnig &rarr; regnerisch &rarr; bewölkt &rarr; sonnig &rarr; bewölkt &rarr; sonnig


## Übergangsmatrix berechnen

Hat man eine Sequenz von Ereignissen gegeben und möchte die zugehörige Markow-Kette berechnen, bzw. die Übergangswahrscheinlichkeiten dieser, so betrachtet man zuerst alle aufeinanderfolgende Paare von Ereignissen. Nun zählt man, wie häufig die Paare auftauchen und normalisiert die passenden Werte, um Wahrscheinlichkeiten zu erhalten.

Gegeben die Sequenz:

1 &rarr; 2 &rarr; 1 &rarr; 3 &rarr; 2 &rarr; 1 &rarr; 3 &rarr; 1 &rarr; 3 &rarr; 2 &rarr; 3

Nun erhalten wir die Paare:

(1,2), (2,1), (1,3), (3,2), (2,1), (1,3), (3,1), (1,3), (3,2), (2,3)

Mit den Häufigkeiten:

(1,2)|(1,3)|(2,1)|(2,3)|(3,1)|(3,2)
---|---|---|---|---|---
1| 3 | 2 | 1 | 1 | 2

Als Matrix:

 Matrix  | 1 | 2 | 3
 ---|:---:|:---:|:---:
**1** | 0 | 1 | 3
**2** |2 | 0 | 1
**3** | 1 | 2 | 0

Auf Wahrscheinlichkeiten normalisiert:

 Matrix  | 1 | 2 | 3
 ---|:---:|:---:|:---:
**1** | 0 | 0.25 | 0.75
**2** |0.67 | 0 | 0.33
**3** | 0.33 | 0.67 | 0

## Ketten n-ter Ordnung

Möchte man nicht nur den derzeitigen Zustand als Basis für den Nächsten nehmen, so kann man Markow-Ketten höherer Ordnung benutzen. Bei einer Markow-Kette n-ter Ordnung werden also die letzten n Ereignisse benutzt, um das nächste Ereigniss vorherzusagen.


# Markow-Ketten für simple Musikstücke

In [6]:
#Erzeugt alle Übergänge n-ter Ordnung aus einer Sequenz
def get_n_transitions(sequence,n):
    transition_list = []
    for idx,target in enumerate(sequence[n:]):
        previous = tuple([sequence[idx+offset] for offset in range(n)])
        transition_list.append((previous,target))
    return transition_list

In [7]:
#Hilsfunktion, welche die Ereignisse auf den Zahlenraum von 0..m kodiert, bzw. dekodiert
def encode_data(transitions):
    encoder = [{},{}]
    decoder = [{},{}]
    
    for idx,previous in enumerate(set([pair[0] for pair in transitions])):
        encoder[0][previous] = idx
        decoder[0][idx] = previous
        
    for idx,target in enumerate(set([pair[1] for pair in transitions])):
        encoder[1][target] = idx
        decoder[1][idx] = target
        
    return encoder, decoder

In [8]:
#Erzeugt die Übergangsmatrix aus Listen von Übergängen
def build_matrix(transitions):
    encoder, decoder = encode_data(transitions)
    
    matrix_shape = (len(encoder[0]),len(encoder[1]))
    
    transition_matrix = lil_matrix(matrix_shape)
    
    for previous,target in transitions:
        previous_idx = encoder[0][previous]
        target_idx = encoder[1][target]
        transition_matrix[previous_idx,target_idx] += 1.
        
    transition_matrix = normalize(transition_matrix,norm='l1',axis=1)
    
    return transition_matrix,encoder,decoder

In [9]:
#Gibt das nächste Ereignis, basierend auf den letzten n Ereignissen zurück
def make_step(previous,transition_matrix,encoder,decoder):
    previous_idx = encoder[0][previous]
    transition_probabilities = transition_matrix[previous_idx].toarray().flatten()
    target_idx = np.random.choice(len(encoder[1]),p=transition_probabilities)
    target = decoder[1][target_idx]
    previous = tuple(list(previous[1:]) + [target])
    return previous,target

In [10]:
#Erzeugt eine Sequenz, generiert mit einer Markow-Kette n-ter Ordnung, 
#basierend auf den gegebenen Trainings Sequenzen
def generate_markow_sequence(train_sequences,n,max_length):
    transitions = []
    for sequence in train_sequences:
        transitions += get_n_transitions(sequence,n)
    
    transition_matrix,encoder,decoder = build_matrix(transitions)
    
    output_sequence = []
    
    previous_idx = np.random.choice(list(decoder[0].keys()))
    previous = decoder[0][previous_idx]
    for note in previous:
        output_sequence.append(note)
    
    for _ in range(max_length):
        previous,note = make_step(previous,transition_matrix,encoder,decoder)
        output_sequence.append(note)
        
        if previous not in encoder[0]:
            break
            
    return output_sequence

Generieren wir nun mehrere "neue" Stücke basierend auf elise für n = 1,3,5,10

In [11]:
note_length = 75
n_ords = [1,3,5,10]
max_length = 300
train_sequences = [midi2note_sequence(elise_midi)]

for n in n_ords:
    new_sequence = generate_markow_sequence(train_sequences,n,max_length)

    midi_out = note_sequence2midi(new_sequence,note_length)
    midi_out.save("./Results/elise_simple_markov_{}.mid".format(n))

# Alle Stücke als Basis

Jetzt nutzen wir alle vorhandenen midi Dateien als Basis für die Markow-Ketten.

In [12]:
train_files = glob.glob("./Data/Beethoven/*.mid")
train_midis = [mido.MidiFile(file) for file in train_files]

note_length = 75
n_ords = [1,3,5,10]
max_length = 300
train_sequences = [midi2note_sequence(midi) for midi in train_midis]

for n in n_ords:
    new_sequence = generate_markow_sequence(train_sequences,n,max_length)

    midi_out = note_sequence2midi(new_sequence,note_length)
    midi_out.save("./Results/beethoven_simple_markov_{}.mid".format(n))

# Erweiterte Darstellung

Um das Timing und gleichzeitige Spielen von Noten abzubilden, betrachten wir die Spuren der linken und rechten Hand und tragen alle Noten in eine gemeinsame Zeitreihe ein.

In [13]:
#Midi in Zeitreihe von Noten umwandeln
#Jeweils mit Zeitdifferenz zur vorhergehenden Note, Notenwert und Notenlänge
def midi2note_timeline(midi):
    tracks = [track for track in midi.tracks if track.name in ['Piano right','Piano left']]
    
    global_time = 0
    track_times = [0 for _ in tracks]
    tracks_done = [False for _ in tracks]
    track_index = [0 for _ in tracks]
    sequence = []
    
    while False in tracks_done:
        next_message = (-1, 10**10)
        for index, track in enumerate(tracks):
            if not tracks_done[index]:
                current_time = track_times[index] + track[track_index[index]].time
                if current_time < next_message[1]:
                    next_message = (index, current_time)
        
        index, current_time = next_message
        if tracks[index][track_index[index]].type == 'note_on':
            sequence.append(vars(tracks[index][track_index[index]]).copy())
            sequence[-1]['time'] = current_time - global_time
        global_time = current_time
        track_times[index] += tracks[index][track_index[index]].time
        track_index[index] += 1
        if track_index[index] >= len(tracks[index]):
            tracks_done[index] = True
    
    currently_played = dict()
    timeline = []
    global_time = 0
    for entry in sequence:
        global_time += entry['time']
        if entry['velocity'] > 0:
            currently_played[entry['note']] = global_time
        else:
            start_time = currently_played.pop(entry['note'],None)
            if start_time is not None:
                timeline.append((start_time,entry['note'],global_time-start_time))

    timeline = sorted(timeline)
    timeline = [(entry[0]-timeline[index-1][0],entry[1],entry[2]) if index > 0 else entry 
                 for index,entry in enumerate(timeline)]
            
    return timeline

In [14]:
#Hier wird eine Zeitreihe von Noten wieder in Midi Daten umgewandelt
def note_timeline2midi(timeline,speed_factor=1.):
    detailed_series = []
    global_time = 0
    for entry in timeline:
        global_time += entry[0]
        detailed_series.append((global_time,entry[1],'note_on'))
        detailed_series.append((global_time+entry[2],entry[1],'note_off'))
        
    detailed_series = sorted(detailed_series)
    
    output_midi = mido.MidiFile()
    track = mido.MidiTrack()
    output_midi.tracks.append(track)
    
    track.append(mido.Message('program_change', program=0))

    global_time = 0
    for event in detailed_series:
        track.append(mido.Message(event[2],note=event[1],
                                  time=int((event[0]-global_time)*speed_factor)))
        global_time = event[0] 

    return output_midi

Jetzt wenden wir diese erweiterte Darstellung einmal auf Für Elise an.

In [15]:
timeline = midi2note_timeline(elise_midi)

midi_out = note_timeline2midi(timeline)
midi_out.save("./Results/elise_detailed.mid")

# Markow-Ketten für die erweiterte Darstellung

Wir erzeugen Stücke basierend auf der erweiterten Darstellung von Für Elise.

In [16]:
n_ords = [1,3,5,10]
max_length = 600
train_sequences = [midi2note_timeline(elise_midi)]

for n in n_ords:
    new_sequence = generate_markow_sequence(train_sequences,n,max_length)

    midi_out = note_timeline2midi(new_sequence)
    midi_out.save("./Results/elise_detailed_markov_{}.mid".format(n))

# Alle Stücke als Basis

Auch für diese Darstellung betrachten wir noch einmal Ergebnisse basierend auf allen Trainingsstücken.

In [17]:
train_files = glob.glob("./Data/Beethoven/*.mid")
train_midis = [mido.MidiFile(file) for file in train_files]

n_ords = [1,3,5,10]
max_length = 600
train_sequences = [midi2note_timeline(midi) for midi in train_midis]

for n in n_ords:
    new_sequence = generate_markow_sequence(train_sequences,n,max_length)

    midi_out = note_timeline2midi(new_sequence)
    midi_out.save("./Results/beethoven_detailed_markov_{}.mid".format(n))

# Markow-Ketten variabler Ordnung

Hier wird die Ordnung zufällig hoch oder runter gesetzt, um mehr Variation in das generierte Stück zu bringen.

In [18]:
#Genereirt Markow-Ketten Übergänge für die Ordnungen 1 bis max_n
#mit diesen Übergängen wird eine Sequenz erzeugt, wobei mit der wahrscheinlichkeit change_probability
#die Ordnung geändert wird
#basierend auf den gegebenen Trainings Sequenzen
def generate_variable_markow_sequence(train_sequences,max_n,max_length,change_probability=0.25):
    matrix_list = []
    encode_list = []
    decode_list = []
    
    for n in range(max_n):
        transitions = []
        for sequence in train_sequences:
            transitions += get_n_transitions(sequence,n+1)
    
        transition_matrix,encoder,decoder = build_matrix(transitions)
        matrix_list.append(transition_matrix)
        encode_list.append(encoder)
        decode_list.append(decoder)
    
    output_sequence = []
    
    n = int(max_n/2)
    
    previous_idx = np.random.choice(list(decode_list[n][0].keys()))
    previous = decode_list[n][0][previous_idx]
    for note in previous:
        output_sequence.append(note)
    
    for _ in range(max_length):
        new_previous,note = make_step(previous,matrix_list[n],encode_list[n],decode_list[n])
        output_sequence.append(note)
        
        if np.random.rand() < change_probability:
            if n == max_n-1 or (np.random.rand() < 0.5 and n != 0):
                n -= 1
                new_previous = tuple(list(new_previous[1:]))
            else:
                n += 1
                new_previous = tuple(list(previous) + [note])
        
        previous = new_previous
        
        if previous not in encode_list[n][0]:
            break
            
    return output_sequence

# Anwendung auf Für Elise

In [19]:
note_length = 75    
max_n_list = [5,10]
max_length = 400
train_sequences = [midi2note_sequence(elise_midi)]

for max_n in max_n_list:
    new_sequence = generate_variable_markow_sequence(train_sequences,max_n,max_length)

    midi_out = note_sequence2midi(new_sequence,note_length)
    midi_out.save("./Results/elise_simple_markov_variable_{}.mid".format(max_n))

In [20]:
max_n_list = [5,10]
max_length = 600
train_sequences = [midi2note_timeline(elise_midi)]

for max_n in max_n_list:
    new_sequence = generate_variable_markow_sequence(train_sequences,max_n,max_length)

    midi_out = note_timeline2midi(new_sequence)
    midi_out.save("./Results/elise_detailed_markov_variable_{}.mid".format(max_n))

# Anwendung auf alle Stücke

In [21]:
train_files = glob.glob("./Data/Beethoven/*.mid")
train_midis = [mido.MidiFile(file) for file in train_files]

note_length = 75  
max_n_list = [5,10]
max_length = 600
train_sequences = [midi2note_sequence(midi) for midi in train_midis]

for max_n in max_n_list:
    new_sequence = generate_variable_markow_sequence(train_sequences,max_n,max_length)

    midi_out = note_sequence2midi(new_sequence,note_length)
    midi_out.save("./Results/beethoven_simple_markov_variable_{}.mid".format(max_n))

In [22]:
train_files = glob.glob("./Data/Beethoven/*.mid")
train_midis = [mido.MidiFile(file) for file in train_files]

max_n_list = [5,10]
max_length = 600
train_sequences = [midi2note_timeline(midi) for midi in train_midis]

for max_n in max_n_list:
    new_sequence = generate_variable_markow_sequence(train_sequences,max_n,max_length)

    midi_out = note_timeline2midi(new_sequence)
    midi_out.save("./Results/beethoven_detailed_markov_variable_{}.mid".format(max_n))